#Book4 Ch9 09-6-3 to 09-6-4

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install xgboost

In [ ]:
pip install lightgbm

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Book4/Ch9/Apart_tree1.csv')

df.shape

(49436, 10)

In [ ]:
df.head(2)

,지역코드,월,전용면적,층,거래금액,거래금액1,연도1,건축년도1,법정동2,아파트2
0,30200,4,84.984,3,29500,10.292179,58,40,174,30
1,41570,9,101.960,11,44500,10.703267,60,51,157,33


In [ ]:
df.drop(['거래금액'], axis=1, inplace=True) #'거래금액' 변수 제외
df.shape

(49436, 9)

In [ ]:
# 참조 코딩
df.head(3)

,지역코드,월,전용면적,층,거래금액1,연도1,건축년도1,법정동2,아파트2
0,30200,4,84.9840,3,10.292179,58,40,174,30
1,41570,9,101.9600,11,10.703267,60,51,157,33
2,26140,12,102.7835,5,10.854199,56,54,28,22


In [ ]:
data = df.drop(['거래금액1'], axis=1)   # 타깃 변수를 제외한 설명 변수를 data에 저장
target = df['거래금액1']                # 타깃 변수만 target에 저장

# 80:20 데이터 변환
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.2, random_state=42)   # test_size=0.2임에 주의

# 결정 트리 모델 및 랜덤 포레스트 모델

In [ ]:
# 디폴트 모델
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
tree = DecisionTreeRegressor(random_state=42)
rf = RandomForestRegressor(random_state=42)

In [ ]:
# Decision Tree 모델 (Default 모델)
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor(random_state=42)
tree.fit(X_train, y_train)

print("Accuracy(GINI) on training set:{:.5f}".format(tree.score(X_train, y_train)))
print("Accuracy(GINI) on test set:{:.5f}".format(tree.score(X_test, y_test)))

Accuracy(GINI) on training set:0.99999
Accuracy(GINI) on test set:0.82026


In [ ]:
# 그리드 서치

from sklearn.model_selection import GridSearchCV
tree = DecisionTreeRegressor(random_state=42)

params = {'max_depth': range(1,21)}

grid_tree = GridSearchCV(tree, param_grid=params, scoring='r2', cv=3, n_jobs=-1,
                     verbose=1)
grid_tree.fit(X_train, y_train)

print("GridSearchCV max accuracy:{:.5f}".format(grid_tree.best_score_))
print("GridSearchCV best parameter:", (grid_tree.best_params_))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
GridSearchCV max accuracy:0.82463
GridSearchCV best parameter: {'max_depth': 12}


In [ ]:
model = grid_tree.best_estimator_
pred = model.predict(X_test)

print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

r2: 0.83801


#랜덤 포레스트 모델

In [ ]:
# Random Forest 모델 (Default 모델)
from sklearn.ensemble import RandomForestRegressor
tree = RandomForestRegressor(random_state=42, n_estimators=50, max_depth=10)
tree.fit(X_train, y_train)

print("Accuracy(GINI) on training set:{:.5f}".format(tree.score(X_train, y_train)))
print("Accuracy(GINI) on test set:{:.5f}".format(tree.score(X_test, y_test)))

Accuracy(GINI) on training set:0.87462
Accuracy(GINI) on test set:0.85201


In [ ]:
### 런타임 1분 40초 소요
from sklearn.model_selection import GridSearchCV
rf = RandomForestRegressor(random_state=42)

# 챗GP가 랜덤 포레스트 모델을 n_estimator=50을 넘어가면 시간 초과 에러가 나서
# 책과 n_estimators 값을 맞추기 위해 파이썬에서도 50으로 고정
params = {'max_depth':[10, 20, 30, 40, 50], 'n_estimators':[50]}

grid_rf = GridSearchCV(rf, param_grid=params, scoring='r2', cv=3, n_jobs=-1,
                     verbose=1)
grid_rf.fit(X_train, y_train)

print("GridSearchCV max accuracy:{:.5f}".format(grid_rf.best_score_))
print("GridSearchCV best parameter:", (grid_rf.best_params_))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
GridSearchCV max accuracy:0.89585
GridSearchCV best parameter: {'max_depth': 30, 'n_estimators': 50}


In [ ]:
# 그리드 서치

from sklearn.model_selection import GridSearchCV
rf = RandomForestRegressor(random_state=42)
params = {'max_depth': range(20, 51, 5), 'n_estimators':[50]}

grid_rf = GridSearchCV(rf, param_grid=params, scoring='r2', cv=3, n_jobs=-1,
                     verbose=1)
grid_rf.fit(X_train, y_train)

print("GridSearchCV max accuracy:{:.5f}".format(grid_rf.best_score_))
print("GridSearchCV best parameter:", (grid_rf.best_params_))

Fitting 3 folds for each of 7 candidates, totalling 21 fits
GridSearchCV max accuracy:0.89585
GridSearchCV best parameter: {'max_depth': 30, 'n_estimators': 50}


In [ ]:
# 그리드 서치

from sklearn.model_selection import GridSearchCV
rf = RandomForestRegressor(random_state=42)
params = {'max_depth': range(26,35), 'n_estimators':[50]}

grid_rf = GridSearchCV(rf, param_grid=params, scoring='r2', cv=3, n_jobs=-1,
                     verbose=1)
grid_rf.fit(X_train, y_train)

print("GridSearchCV max accuracy:{:.5f}".format(grid_rf.best_score_))
print("GridSearchCV best parameter:", (grid_rf.best_params_))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
GridSearchCV max accuracy:0.89601
GridSearchCV best parameter: {'max_depth': 26, 'n_estimators': 50}


In [ ]:
# 최종 그리드 서치
# 런타임 1분 40초 소요
from sklearn.model_selection import GridSearchCV
rf = RandomForestRegressor(random_state=42)
params = {'max_depth': range(21,27), 'n_estimators':[50]}

grid_rf = GridSearchCV(rf, param_grid=params, scoring='r2', cv=3, n_jobs=-1,
                     verbose=1)
grid_rf.fit(X_train, y_train)

print("GridSearchCV max accuracy:{:.5f}".format(grid_rf.best_score_))
print("GridSearchCV best parameter:", (grid_rf.best_params_))

Fitting 3 folds for each of 6 candidates, totalling 18 fits
GridSearchCV max accuracy:0.89601
GridSearchCV best parameter: {'max_depth': 26, 'n_estimators': 50}


In [ ]:
rf = RandomForestRegressor(random_state=42)
model = grid_rf.best_estimator_
pred = model.predict(X_test)

print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

r2: 0.90605


# 그레이디언트 부스팅 모델

In [ ]:
# Gradient Boosting 모델 (Default 모델)
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
# 책의 챗GPT에서 실행되는 그레이디언트 부스팅 모델과 파라미터 동일하게 설정
gr = GradientBoostingRegressor(random_state = 42, n_estimators=30, max_depth=10)
gr.fit(X_train, y_train)
pred = gr.predict(X_test)

print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

r2: 0.89151


In [ ]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV

# n_estimators는 GradientBoostingRegressor 디폴트 값 100으로 복귀
gr = GradientBoostingRegressor(random_state = 42)

parameters = {'learning_rate': [0.1],
              'max_depth': [5, 10, 15, 20],
              'n_estimators': [100],
              'subsample': [0.9]
              }

grid_gr = GridSearchCV(gr,
                        parameters,
                        scoring = 'r2',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_gr.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=3, estimator=GradientBoostingRegressor(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1], 'max_depth': [5, 10, 15, 20],
                         'n_estimators': [100], 'subsample': [0.9]},
             scoring='r2', verbose=True)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_gr.best_params_)

GridSearchCV 최적 파라미터: {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 100, 'subsample': 0.9}


In [ ]:
model = grid_gr.best_estimator_
pred = model.predict(X_test)
print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

r2: 0.92051


In [ ]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV

gr = GradientBoostingRegressor(random_state=42)

parameters = {'learning_rate': [0.1],
              'max_depth': [6, 7, 8, 9, 10],
              'n_estimators': [100],
              'subsample': [0.9]
              }

grid_gr = GridSearchCV(gr,
                        parameters,
                        scoring = 'r2',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_gr.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


GridSearchCV(cv=3, estimator=GradientBoostingRegressor(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1], 'max_depth': [6, 7, 8, 9, 10],
                         'n_estimators': [100], 'subsample': [0.9]},
             scoring='r2', verbose=True)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_gr.best_params_)

GridSearchCV 최적 파라미터: {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 100, 'subsample': 0.9}


In [ ]:
model = grid_gr.best_estimator_
pred = model.predict(X_test)
print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

r2: 0.92051


In [ ]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV

gr = GradientBoostingRegressor(random_state = 42)

parameters = {'learning_rate': [0.1],
              'max_depth': [10, 11, 12, 13, 14],
              'n_estimators': [100],
              'subsample': [0.9]
              }

gr_grid = GridSearchCV(gr,
                        parameters,
                        scoring = 'r2',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
gr_grid.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


GridSearchCV(cv=3, estimator=GradientBoostingRegressor(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1],
                         'max_depth': [10, 11, 12, 13, 14],
                         'n_estimators': [100], 'subsample': [0.9]},
             scoring='r2', verbose=True)

In [ ]:
print('GridSearchCV 최적 파라미터:', gr_grid.best_params_)

GridSearchCV 최적 파라미터: {'learning_rate': 0.1, 'max_depth': 11, 'n_estimators': 100, 'subsample': 0.9}


In [ ]:
model = gr_grid.best_estimator_
pred = model.predict(X_test)
print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

r2: 0.92191


In [ ]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV

gr = GradientBoostingRegressor(random_state = 42)

parameters = {'learning_rate': [0.1],
              'max_depth': [11],
              'n_estimators': [100],
              'subsample': [0.7, 0.8, 0.9, 1]
              }

grid_gr = GridSearchCV(gr,
                        parameters,
                        scoring = 'r2',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_gr.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=3, estimator=GradientBoostingRegressor(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1], 'max_depth': [11],
                         'n_estimators': [100],
                         'subsample': [0.7, 0.8, 0.9, 1]},
             scoring='r2', verbose=True)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_gr.best_params_)

GridSearchCV 최적 파라미터: {'learning_rate': 0.1, 'max_depth': 11, 'n_estimators': 100, 'subsample': 0.9}


In [ ]:
model = grid_gr.best_estimator_
pred = model.predict(X_test)
print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

r2: 0.92191


In [ ]:
# 최종 그리드 서치
# 런타임 1분 50초 소요
from sklearn.model_selection import GridSearchCV

gr = GradientBoostingRegressor(random_state = 42)

parameters = {'learning_rate': [0.01, 0.05, 0.1, 0.02],
              'max_depth': [11],
              'n_estimators': [100],
              'subsample': [0.9]
              }

grid_gr = GridSearchCV(gr,
                        parameters,
                        scoring = 'r2',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_gr.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=3, estimator=GradientBoostingRegressor(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1, 0.02],
                         'max_depth': [11], 'n_estimators': [100],
                         'subsample': [0.9]},
             scoring='r2', verbose=True)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_gr.best_params_)

GridSearchCV 최적 파라미터: {'learning_rate': 0.1, 'max_depth': 11, 'n_estimators': 100, 'subsample': 0.9}


In [ ]:
model = grid_gr.best_estimator_
pred = model.predict(X_test)
print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

r2: 0.92191


# XGBoost 모델

In [ ]:
# 기본 XGBRegressor 모델
from sklearn.metrics import r2_score

xgb = XGBRegressor(random_state=42)
xgb.fit(X_train, y_train)
pred = xgb.predict(X_test)

print('r2: {0:.5f}'.format(r2_score(y_test, pred)))


r2: 0.91379


In [ ]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV

xgb = XGBRegressor(random_state=42)

parameters = {'colsample_bytree': [1],
              'learning_rate': [0.1],
              'max_depth': [5, 10, 15, 20],
              'min_child_weight' : [1],
              'n_estimators': [100],
              'subsample': [1]
              }

grid_xgb = GridSearchCV(xgb,
                        parameters,
                        scoring = 'r2',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_xgb.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=42, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [1], 'learning_rate': [0.1],
                         'max_depth': [5, 10, 15, 20], 'min_child_weight': [1],
                         'n_estimators': [100], 'subsample': [1]},
             scoring='r2', verbose=True)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_xgb.best_params_)

GridSearchCV 최적 파라미터: {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 10, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1}


In [ ]:
model = grid_xgb.best_estimator_
pred = model.predict(X_test)

print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

r2: 0.91852


In [ ]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV

xgb = XGBRegressor(random_state=42)

parameters = {'colsample_bytree': [1],
              'learning_rate': [0.1],
              'max_depth': [6, 7, 8, 9, 10],
              'min_child_weight' : [1],
              'n_estimators': [100],
              'subsample': [1]
              }

grid_xgb = GridSearchCV(xgb,
                        parameters,
                        scoring = 'r2',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_xgb.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=42, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [1], 'learning_rate': [0.1],
                         'max_depth': [6, 7, 8, 9, 10], 'min_child_weight': [1],
                         'n_estimators': [100], 'subsample': [1]},
             scoring='r2', verbose=True)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_xgb.best_params_)

GridSearchCV 최적 파라미터: {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 10, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1}


In [ ]:
model = grid_xgb.best_estimator_
pred = model.predict(X_test)

print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

r2: 0.91852


In [ ]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV

xgb = XGBRegressor(random_state=42)

parameters = {'colsample_bytree': [1],
              'learning_rate': [0.1],
              'max_depth': [10, 11, 12, 13, 14],
              'min_child_weight': [1],
              'n_estimators': [100],
              'subsample': [1]
              }

grid_xgb = GridSearchCV(xgb,
                        parameters,
                        scoring = 'r2',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_xgb.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=42, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [1], 'learning_rate': [0.1],
                         'max_depth': [10, 11, 12, 13, 14],
                         'min_child_weight': [1], 'n_estimators': [100],
                         'subsample': [1]},
             scoring='r2', verbose=True)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_xgb.best_params_)

GridSearchCV 최적 파라미터: {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 11, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1}


In [ ]:
model = grid_xgb.best_estimator_
pred = model.predict(X_test)

print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

r2: 0.91978


In [ ]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV

xgb = XGBRegressor(random_state=42)

parameters = {'colsample_bytree': [0.7, 0.8, 0.9, 1],
              'learning_rate': [0.1],
              'max_depth': [11],
              'min_child_weight' : [1],
              'n_estimators': [100],
              'subsample': [0.7, 0.8, 0.9, 1]
              }

grid_xgb = GridSearchCV(xgb,
                        parameters,
                        scoring = 'r2',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_xgb.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=42, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.7, 0.8, 0.9, 1],
                         'learning_rate': [0.1], 'max_depth': [11],
                         'min_child_weight': [1], 'n_estimators': [100],
                         'subsample': [0.7, 0.8, 0.9, 1]},
             scoring='r2', verbose=True)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_xgb.best_params_)

GridSearchCV 최적 파라미터: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 11, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.9}


In [ ]:
model = grid_xgb.best_estimator_
pred = model.predict(X_test)

print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

r2: 0.92287


In [ ]:
# 최종 그리드 서치
# 런타임 1분 30초 소요
from sklearn.model_selection import GridSearchCV

xgb = XGBRegressor(random_state=42)

parameters = {'colsample_bytree': [0.8, 0.9],
              'learning_rate': [0.01, 0.05, 0.1, 0.2],
              'max_depth': [11],
              'min_child_weight' : [1],
              'n_estimators': [100],
              'subsample': [0.8, 0.9]
              }

grid_xgb = GridSearchCV(xgb,
                        parameters,
                        scoring = 'r2',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_xgb.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=42, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 0.9],
                         'learning_rate': [0.01, 0.05, 0.1, 0.2],
                         'max_depth': [11], 'min_child_weight': [1],
                         'n_estimators': [100], 'subsample': [0.8, 0.9]},
             scoring='r2', verbose=True)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_xgb.best_params_)

GridSearchCV 최적 파라미터: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 11, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.9}


In [ ]:
model = grid_xgb.best_estimator_
pred = model.predict(X_test)

print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

r2: 0.92287


# 6.4.3 LightGBM 모델

In [ ]:
# 기본 lGBMRegressor 모델
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score

lgb = LGBMRegressor(random_state=42)
lgb.fit(X_train, y_train)
pred = lgb.predict(X_test)

print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001123 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 855
[LightGBM] [Info] Number of data points in the train set: 39548, number of used features: 8
[LightGBM] [Info] Start training from score 10.042557
r2: 0.88972


In [ ]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

lgb = LGBMRegressor(random_state=42)

# 워닝 메시지에서 2^max_depth > num_leaves를 권고해서 num_leaves 값을 1000로 정함
parameters = {'colsample_bytree': [0.7, 0.8, 0.9, 1],
              'learning_rate': [0.1],
              'max_depth': [5, 10],
              'num_leaves': [1000],
              'min_child_weight' : [1],
              'n_estimators': [1000],
              'subsample': [0.7, 0.8, 0.9, 1]
              }

grid_lgb = GridSearchCV(lgb,
                        parameters,
                        scoring = 'r2',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_lgb.fit(X_train, y_train)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_lgb.best_params_)

GridSearchCV 최적 파라미터: {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 10, 'min_child_weight': 1, 'n_estimators': 100, 'num_leaves': 1024, 'subsample': 0.7}


In [ ]:
from sklearn.metrics import r2_score

model = grid_lgb.best_estimator_
pred = model.predict(X_test)

print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

r2: 0.91686


In [ ]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

lgb = LGBMRegressor(random_state=42)

parameters = {'colsample_bytree': [0.7],
              'learning_rate': [0.01],
              'max_depth': [5, 10, 15],
              'num_leaves': [2000],
              'min_child_weight' : [1],
              'n_estimators': [1000],
              'subsample': [0.7]
              }
grid_lgb = GridSearchCV(lgb,
                        parameters,
                        scoring = 'r2',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_lgb.fit(X_train, y_train)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_lgb.best_params_)

GridSearchCV 최적 파라미터: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 15, 'min_child_weight': 1, 'n_estimators': 1000, 'num_leaves': 2000, 'subsample': 0.7}


In [ ]:
from sklearn.metrics import r2_score
model = grid_lgb.best_estimator_
pred = model.predict(X_test)
print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

r2: 0.92706


In [ ]:
# 그리드 서치
# 주의! 런타임 7분 소요
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

lgb = LGBMRegressor(random_state=42)

parameters = {'colsample_bytree': [0.7],
              'learning_rate': [0.01],
              'max_depth': [15, 20, 25, 30],
              'num_leaves': [2000],
              'min_child_weight' : [1],
              'n_estimators': [1000],
              'subsample': [0.7]
              }

grid_lgb = GridSearchCV(lgb,
                        parameters,
                        scoring = 'r2',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_lgb.fit(X_train, y_train)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_lgb.best_params_)

GridSearchCV 최적 파라미터: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 30, 'min_child_weight': 1, 'n_estimators': 1000, 'num_leaves': 2000, 'subsample': 0.7}


In [ ]:
from sklearn.metrics import r2_score
model = grid_lgb.best_estimator_
pred = model.predict(X_test)
print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

r2: 0.92865


In [ ]:
# 최종 그리드 서치
# 주의! 런타임 6분 소요

# 이 책의 분석에서는 max_depth의 최대값을 50으로 제한함. 그러나 독자 여러분은 여기서 더 높은 수치를 시도해 봐도 됩니다.
# 단 그런 경우 시간이 더 많이 소요됨에 주의하세요. 아울러 너무 높은 max_depth는 과적합의 위험이 있음에 유의하세요.
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

lgb = LGBMRegressor(random_state=42)

parameters = {'colsample_bytree': [0.7],
              'learning_rate': [0.01],
              'max_depth': [30, 40, 50],
              'num_leaves': [2000],
              'min_child_weight' : [1],
              'n_estimators': [1000],
              'subsample': [0.7]
              }

grid_lgb = GridSearchCV(lgb,
                        parameters,
                        scoring = 'r2',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
grid_lgb.fit(X_train, y_train)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_lgb.best_params_)

GridSearchCV 최적 파라미터: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 50, 'min_child_weight': 1, 'n_estimators': 1000, 'num_leaves': 2000, 'subsample': 0.7}


In [ ]:
from sklearn.metrics import r2_score
model = grid_lgb.best_estimator_
pred = model.predict(X_test)
print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

r2: 0.92870
